# Summary

* This is a tutorial on using Python for accessing the scBaseCamp dataset hosted by the Arc Institute.
* The data can be streamed or downloaded locally.
  * For small jobs (e.g., summarizing the some metadata), streaming is recommended.
  * For large jobs (e.g., training a model), downloading is recommended.
* See the [README](README.md#metadata) for a description of the obs metadata.


# Setup

### Installation

If needed, install the necessary dependencies.

You can use the [conda environment](../conda_envs/python.yml) provided in this git repository. To do so:

In [ ]:
!which conda && conda env create -q -f ../conda_envs/python.yml

# Load packages

In [1]:
import os
import pandas as pd
import scanpy as sc
import pyarrow.dataset as ds
import gcsfs

In [2]:
# initialize GCS file system for reading data from GCS
fs = gcsfs.GCSFileSystem()

# Data location

In [3]:
# GCS bucket path
gcs_base_path = "gs://arc-ctc-scbasecamp/2025-02-25/"

In [4]:
# STARsolo feature type
feature_type = "GeneFull_Ex50pAS"

# List available files

Let's see what we have to work with!

In [5]:
# helper function to list files 
def get_file_table(gcs_base_path: str, target: str=None, endswith: str=None):
    files = fs.glob(os.path.join(gcs_base_path, "**"))
    if target:
        files = [f for f in files if os.path.basename(f) == target]
    else:
        files = [f for f in files if f.endswith(endswith)]
    file_list = []
    for f in files:
        file_list.append(f.split("/")[-2:-1] + [f])
    return pd.DataFrame(file_list, columns=["organism", "file_path"])

## Parquet files

* Contain the obs metadata
* These can be read efficiently with [pyarrow](https://arrow.apache.org/docs/python/index.html)
  * We will read in via pyarrow and convert to pandas

In [6]:
# set the path to the metadata files
gcs_path = os.path.join(gcs_base_path, "metadata", feature_type)
gcs_path

'gs://arc-ctc-scbasecamp/2025-02-25/metadata/GeneFull_Ex50pAS'

### List per-sample metadata files

Per-sample (SRX accession) metadata (e.g., tissue)

In [7]:
# list files
sample_pq_files = get_file_table(gcs_path, "sample_metadata.parquet")
print(sample_pq_files.shape)
sample_pq_files.head()

(21, 2)


,organism,file_path
0,Arabidopsis_thaliana,arc-ctc-scbasecamp/2025-02-25/metadata/GeneFul...
1,Bos_taurus,arc-ctc-scbasecamp/2025-02-25/metadata/GeneFul...
2,Caenorhabditis_elegans,arc-ctc-scbasecamp/2025-02-25/metadata/GeneFul...
3,Callithrix_jacchus,arc-ctc-scbasecamp/2025-02-25/metadata/GeneFul...
4,Danio_rerio,arc-ctc-scbasecamp/2025-02-25/metadata/GeneFul...


**Notes:**

* As you can see, the files are organized by `feature_type` (STARsolo output type) and `organism`

### List per-obs metadata files

Per-observation (cell) metadata

In [8]:
# list files
obs_pq_files = get_file_table(gcs_path, "obs_metadata.parquet")
print(obs_pq_files.shape)
obs_pq_files.head()

(21, 2)


,organism,file_path
0,Arabidopsis_thaliana,arc-ctc-scbasecamp/2025-02-25/metadata/GeneFul...
1,Bos_taurus,arc-ctc-scbasecamp/2025-02-25/metadata/GeneFul...
2,Caenorhabditis_elegans,arc-ctc-scbasecamp/2025-02-25/metadata/GeneFul...
3,Callithrix_jacchus,arc-ctc-scbasecamp/2025-02-25/metadata/GeneFul...
4,Danio_rerio,arc-ctc-scbasecamp/2025-02-25/metadata/GeneFul...


## h5ad files 

* Contain count matrices and per-obs metadata

In [9]:
# set the path
gcs_path = os.path.join(gcs_base_path, "h5ad", feature_type)
gcs_path

'gs://arc-ctc-scbasecamp/2025-02-25/h5ad/GeneFull_Ex50pAS'

In [10]:
# list files
h5ad_files = get_file_table(gcs_path, endswith=".h5ad")
print(h5ad_files.shape)
h5ad_files.head()

(30387, 2)


,organism,file_path
0,Arabidopsis_thaliana,arc-ctc-scbasecamp/2025-02-25/h5ad/GeneFull_Ex...
1,Arabidopsis_thaliana,arc-ctc-scbasecamp/2025-02-25/h5ad/GeneFull_Ex...
2,Arabidopsis_thaliana,arc-ctc-scbasecamp/2025-02-25/h5ad/GeneFull_Ex...
3,Arabidopsis_thaliana,arc-ctc-scbasecamp/2025-02-25/h5ad/GeneFull_Ex...
4,Arabidopsis_thaliana,arc-ctc-scbasecamp/2025-02-25/h5ad/GeneFull_Ex...


# Explore the per-sample metadata

### Just human samples

In [11]:
# get the per-sample metadata file path
infile = sample_pq_files[sample_pq_files["organism"] == "Homo_sapiens"]["file_path"].values[0]
infile

'arc-ctc-scbasecamp/2025-02-25/metadata/GeneFull_Ex50pAS/Homo_sapiens/sample_metadata.parquet'

In [12]:
# load the metadata
sample_metadata = ds.dataset(infile, filesystem=fs, format="parquet").to_table().to_pandas()
print(sample_metadata.shape)
sample_metadata.head()

(16077, 14)


,entrez_id,srx_accession,file_path,obs_count,lib_prep,tech_10x,cell_prep,organism,tissue,disease,perturbation,cell_line,czi_collection_id,czi_collection_name
0,29110018,ERX11148735,gs://arc-ctc-scbasecamp/2025-02-25/h5ad/GeneFu...,747,10x_Genomics,3_prime_gex,single_cell,Homo sapiens,skin of body,normal,surplus skin from breast reconstruction surgery,not applicable,73f82ac8-15cc-4fcd-87f8-5683723fce7f,Developmental cell programs are co-opted in in...
1,29110027,ERX11148744,gs://arc-ctc-scbasecamp/2025-02-25/h5ad/GeneFu...,2379,10x_Genomics,3_prime_gex,single_cell,Homo sapiens,skin of body,normal,treated with dispase II and collagenase for ce...,keratinocyte CD49f-,73f82ac8-15cc-4fcd-87f8-5683723fce7f,Developmental cell programs are co-opted in in...
2,29110026,ERX11148743,gs://arc-ctc-scbasecamp/2025-02-25/h5ad/GeneFu...,2316,10x_Genomics,3_prime_gex,single_cell,Homo sapiens,skin of body,normal,treated with dispase II and collagenase for ce...,epidermal myeloid cells,73f82ac8-15cc-4fcd-87f8-5683723fce7f,Developmental cell programs are co-opted in in...
3,29110023,ERX11148740,gs://arc-ctc-scbasecamp/2025-02-25/h5ad/GeneFu...,2907,10x_Genomics,3_prime_gex,single_cell,Homo sapiens,skin of body,normal,skin collected from breast reconstruction surgery,not specified,73f82ac8-15cc-4fcd-87f8-5683723fce7f,Developmental cell programs are co-opted in in...
4,29110015,ERX11148732,gs://arc-ctc-scbasecamp/2025-02-25/h5ad/GeneFu...,4082,10x_Genomics,3_prime_gex,single_cell,Homo sapiens,skin of body,normal,treated with dispase II and collagenase,not_applicable,73f82ac8-15cc-4fcd-87f8-5683723fce7f,Developmental cell programs are co-opted in in...


In [18]:
df = sample_metadata

In [19]:
print(df[['entrez_id', 'srx_accession']].head(2))

   entrez_id srx_accession
0   29110018   ERX11148735
1   29110027   ERX11148744


In [13]:
# All human?
sample_metadata["organism"].value_counts()

organism
Homo sapiens    16077
Name: count, dtype: int64

In [14]:
# 10X library prep methods
sample_metadata["tech_10x"].value_counts()

tech_10x
3_prime_gex          10851
5_prime_gex           3746
vdj                    437
multiome               366
not_applicable         250
feature_barcoding      230
other                  168
cellplex                19
flex                     6
atac                     4
Name: count, dtype: int64

In [16]:
# cell prep method
sample_metadata["cell_prep"].value_counts()

cell_prep
single_cell       14661
single_nucleus     1393
unsure               22
not_applicable        1
Name: count, dtype: int64

## Get GEO datasets for each